## Building a semantic search RAG pipeline (Naive RAG)

Welcome! In this notebook, you'll learn how to build a semantic search pipeline using the Haystack framework. We'll walk through each step, from initializing components to running a Retrieval-Augmented Generation (RAG) pipeline that answers questions using indexed documents.


### 1 Imports

Let's start by importing the necessary Haystack components and initializing the core elements of our semantic search pipeline. We'll use a text embedder, a retriever, a prompt builder, and an LLM generator. These components will work together to process queries and generate answers based on your indexed data.


In [ ]:
# Continue from the previous script, assuming 'document_store' is populated.
from scripts.indexing import document_store  # Adjust the import as necessary

# Import necessary components for the query pipeline
from haystack.components.embedders import SentenceTransformersTextEmbedder
from haystack.components.retrievers.in_memory import InMemoryEmbeddingRetriever
from haystack.components.builders import PromptBuilder
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
from haystack import Pipeline


## 2. Building the Naive RAG Pipeline

Now, we'll assemble our pipeline. The pipeline connects the components so that a user's question is embedded, relevant documents are retrieved, a prompt is built, and an answer is generated by the LLM. This forms the backbone of a basic RAG (Retrieval-Augmented Generation) system.

In [6]:

# --- 1. Initialize Query Pipeline Components ---

# Text Embedder: To embed the user's query. Must be compatible with the document embedder.
text_embedder = SentenceTransformersTextEmbedder(model="sentence-transformers/all-MiniLM-L6-v2")

# Retriever: Fetches documents from the DocumentStore based on vector similarity.
retriever = InMemoryEmbeddingRetriever(document_store=document_store, top_k=3)

# PromptBuilder: Creates a prompt using the retrieved documents and the query.
# The Jinja2 template iterates through the documents and adds their content to the prompt.
prompt_template_for_pipeline = """
Given the following information, answer the user's question.
If the information is not available in the provided documents, say that you don't have enough information to answer.

Context:
{% for doc in documents %}
    {{ doc.content }}
{% endfor %}

Question: {{question}}
Answer:
"""
prompt_builder_inst = PromptBuilder(template=prompt_template_for_pipeline,
                                    required_variables="*")
llm_generator_inst = OpenAIGenerator(api_key=Secret.from_env_var("OPENAI_API_KEY"), model="gpt-4o-mini")


# --- 2. Build the Naive RAG Pipeline ---

naive_rag_pipeline = Pipeline()

# Add components to the pipeline
naive_rag_pipeline.add_component("text_embedder", text_embedder)
naive_rag_pipeline.add_component("retriever", retriever)
naive_rag_pipeline.add_component("prompt_builder", prompt_builder_inst)
naive_rag_pipeline.add_component("llm", llm_generator_inst)

# --- 3. Connect the Components ---

# The query embedding is sent to the retriever
naive_rag_pipeline.connect("text_embedder.embedding", "retriever.query_embedding")
# The retriever's documents are sent to the prompt builder
naive_rag_pipeline.connect("retriever.documents", "prompt_builder.documents")
# The final prompt is sent to the LLM
naive_rag_pipeline.connect("prompt_builder.prompt", "llm.prompt")


🚅 Components
  - text_embedder: SentenceTransformersTextEmbedder
  - retriever: InMemoryEmbeddingRetriever
  - prompt_builder: PromptBuilder
  - llm: OpenAIGenerator
🛤️ Connections
  - text_embedder.embedding -> retriever.query_embedding (list[float])
  - retriever.documents -> prompt_builder.documents (list[Document])
  - prompt_builder.prompt -> llm.prompt (str)

## 3. Visualizing the Pipeline

It's helpful to visualize the pipeline to understand how data flows between components. The diagram below shows the connections between each part of our semantic search pipeline.


In [2]:
# --- 4. Visualize the Pipeline ---
naive_rag_pipeline.draw(path="./images/naive_rag_pipeline.png")

![](./images/naive_rag_pipeline.png)

## 4. Running the Pipeline

Let's put our pipeline to the test! We'll ask a question, and the pipeline will retrieve relevant documents and generate an answer. You can modify the question to experiment with different queries.


In [3]:
# --- 5. Run the Pipeline ---

question = "Which company released the Claude 3 model family?"

# The run method requires inputs for the components that don't have an incoming connection.
# In this case, 'text_embedder' needs the 'text' (the query) and 'prompt_builder' needs the 'question'.
result = naive_rag_pipeline.run({
    "text_embedder": {"text": question},
    "prompt_builder": {"question": question}
})

print(f"\nQuestion: {question}")
print(f"Answer: {result['llm']['replies']}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  5.32it/s]



Question: Which company released the Claude 3 model family?
Answer: ['The Claude 3 model family was released by Anthropic.']


Try additional questions to see how the pipeline responds using different sources of indexed data, such as web content or CSV files.


In [4]:
# Another example question using the web data
question_2 = "What is Haystack 2.0?"
result_2 = naive_rag_pipeline.run({
    "text_embedder": {"text": question_2},
    "prompt_builder": {"question": question_2}
})
print(f"\nQuestion: {question_2}")
print(f"Answer: {result_2['llm']['replies']}")

Batches: 100%|██████████| 1/1 [00:00<00:00, 69.67it/s]



Question: What is Haystack 2.0?
Answer: ['Haystack 2.0 is an open-source Python framework for building production-ready LLM (Large Language Model) applications. It allows for the implementation of composable AI systems that are easy to use, customize, extend, optimize, evaluate, and deploy to production. This version is a major rework of the previous version, with the goal of providing flexibility and a common component interface for seamless interaction between different components. Haystack 2.0 integrates with almost all major model providers and databases and enables users to create custom components and foster an open ecosystem around its framework.']


You can also ask questions that target structured data, like information from CSV files. This demonstrates the flexibility of the pipeline in handling various document types.


In [5]:
# Another example question using the csv data
question_2 = "When was Gemini released?"
result_2 = naive_rag_pipeline.run({
    "text_embedder": {"text": question_2},
    "prompt_builder": {"question": question_2}
})
print(f"\nQuestion: {question_2}")
print(f"Answer: {result_2['llm']['replies']}")

Batches: 100%|██████████| 1/1 [00:00<00:00,  7.62it/s]



Question: When was Gemini released?
Answer: ['Gemini was released in 2023.']
